Анастасия Плавина

### Тема: Взаимосвязь величин. Параметрические и непараметрические показатели корреляции. Корреляционный анализ.

In [1]:
import numpy as np
from scipy import stats 

#### Задача 1

Даны значения величины заработной платы заемщиков банка (salary) и значения их поведенческого кредитного скоринга (scoring):

salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) найдите:
1. ковариацию (смещённую и несмещённую) этих двух величин,
2. коэффициент корреляции Пирсона.

Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

In [2]:
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110] 
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

n = len(salary)

Вычисляем смещенную ковариацию по формуле: $\sigma_{XY} = \dfrac{1}{n} \displaystyle\sum_{i = 1}^n (x_i - \overline{X}) \cdot (y_i - \overline{Y})$

Несмещенную: $\sigma_{XY} = \dfrac{1}{n - 1} \displaystyle\sum_{i = 1}^n (x_i - \overline{X}) \cdot (y_i - \overline{Y})$, где X - значения заработной платы заемщиков, а Y - значение скоринга.

In [3]:
def covariance(X, Y, ddof=0):
    res = 0
    n = len(X)
    for i in range(n):
        devx = X[i] - sum(X)/n
        devy = Y[i] - sum(Y)/n
        res += devx * devy
    return res / (n - ddof)

In [4]:
 # смещенная оценка ковариации
covariance(salary, scoring)

9157.84

In [5]:
# несмещенная оценка ковариации
covariance(salary, scoring, ddof=1)

10175.377777777778

In [6]:
# проверка
cov = np.cov(salary, scoring, ddof=0)[0][1] 
cov_unbiased = np.cov(salary, scoring, ddof=1)[0][1] 

cov, cov_unbiased 

(9157.84, 10175.377777777776)

Вычислим коэффицент Пирсона по формуле:
$$r_{XY} = \dfrac{\sigma_{XY}}{\sigma_X \cdot \sigma_Y}$$

In [7]:
def corr_pearson(X, Y):
    devx = 0
    devy = 0
    n = len(X)
    for i in range(n):
        devx += (X[i] - sum(X)/n)**2
        devy += (Y[i] - sum(Y)/n)**2
    stdx = (devx/n)**(1/2)
    stdy = (devy/n)**(1/2)
    
    return  covariance(X, Y) / (stdx * stdy)

In [8]:
r = corr_pearson(salary, scoring)
r

0.8874900920739162

In [9]:
# проверка
np.corrcoef(salary, scoring)[0][1]

0.8874900920739162

#### Задача 2

Проведите тест на значимость коэффициента корреляции Пирсона, найденного в предыдущей задаче. Что для этого нужно знать:
* Нулевая гипотеза: реальный коэффициент корреляции равен 0. Альтернативная гипотеза двухсторонняя.
* Статистика: t = r * sqrt(n - 2) / sqrt(1 - r ** 2), где r - коэффициент корреляции Пирсона, посчитанный по выборке.
* В предположении верности нулевой гипотезы эта статистика имеет распределение Стьюдента с параметром df = n - 2.

H0: коэффициент корреляции равен 0.

H1: коэффициент больше или меньше 0.

Используем следующую статистику для проверки коэффицента корреляции:
$$t = \frac{r \cdot \sqrt{n - 2}} {\sqrt{1 - r^2}}$$

In [10]:
def t_stat(r, n):
    return r * (n - 2)**(1/2) / (1 - r**2)**(1/2)

t = t_stat(r, n)
t

5.447168150485575

Проверим гипотезу при aplha = 0.01. Определим критическую область:

In [11]:
alpha = 0.01

t1 = stats.t.ppf(alpha/2, df = n-2)
t2 = stats.t.ppf(1-alpha/2, df = n-2)

t1, t2

(-3.3553873313333966, 3.3553873313333957)

Значение статистики попадает в критическую область => не можем принять гипотезу H0.
Посчитаем, при каком p-value мы не попадаем в критическую область:

In [12]:
def p_value(t, n):
    p_left = stats.t.cdf(t, df = n-2)
    p_right = 1 - stats.t.cdf(t, df = n-2)
    return 2 * min(p_left, p_right)

pvalue = p_value(t, n)
pvalue

0.0006107546587257318

Проверим:

In [13]:
alpha = pvalue

t1 = stats.t.ppf(alpha/2, df = n-2)
t2 = stats.t.ppf(1-alpha/2, df = n-2)

t1, t2

(-5.447168150556778, 5.447168150556778)

 Т.к. мы не попадаем в крит. обалсть при p-value < 0.05, то можем отвернуть H0.


#### Задача 3

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:

131, 125, 115, 122, 131, 115, 107, 99, 125, 111

Известно, что в генеральной совокупности IQ распределен нормально. Найдите доверительный интервал для математического ожидания с надежностью 0.95.

In [14]:
students = [131, 125, 115, 122, 131, 115, 107, 99, 125, 111]

n = len(students)
MX = sum(students)/n
sigma = np.std(students)
p = 0.95
alpha = 1 - p

1. Определим квантили распределения Cтьюдента:

In [15]:
t1 = stats.t.ppf(alpha/2, df=n-1)
t2 = stats.t.ppf(1-alpha/2, df=n-1)

t1, t2

(-2.2621571627409915, 2.2621571627409915)

2. Определим доверительный интервал по формуле:

$P \left( \overline{X} + t_{\alpha / 2} \cdot \dfrac{\sigma_X}{\sqrt{n}} \leq \mu \leq \overline{X} + t_{1 - \alpha / 2} \cdot \dfrac{\sigma_X}{\sqrt{n}} \right) = p$
    </div>

In [16]:
def confidence_interval(n, MX, sigma, t1, t2):
    left = MX + t1 * sigma / n**(1/2)
    right = MX + t2 * sigma / n**(1/2)
    return left, right

In [17]:
confidence_interval(n, MX, sigma, t1, t2)

(110.9432125583706, 125.25678744162938)

Математическое ожидание IQ с вероятностью 0.95 находится в интервале от 110 до 125.